In [12]:
import pandas as pd
from sqlalchemy import create_engine, text

In [13]:
mysql_user = "user"
mysql_password = "password"
mysql_host = "127.0.0.1"
mysql_port = "3306"
mysql_db = "testdb"

In [14]:
engine = create_engine(f"mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_db}")

In [15]:
csv_file = "datos.csv" # Ruta ao teu arquivo CSV
df = pd.read_csv("/data/netflix_titles.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/data/netflix_titles.csv'

In [ ]:
table_name = "mi_tabla"

In [ ]:
df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

print(f"Datos cargados na táboa '{table_name}' correctamente.")

NameError: name 'df' is not defined

In [ ]:
with engine.connect() as conn:
    # Contar filas
    result = conn.execute(text(f"SELECT COUNT(*) FROM {table_name};"))
    count = result.scalar()
    print(f"Número de filas cargadas: {count}")

    # Mostrar as primeiras 5 filas
    result = conn.execute(text(f"SELECT * FROM {table_name} LIMIT 5;"))
    for row in result:
        print(row)

DatabaseError: (mysql.connector.errors.DatabaseError) 2003 (HY000): Can't connect to MySQL server on '127.0.0.1:3306' (111)
(Background on this error at: https://sqlalche.me/e/20/4xp6)